<a href="https://colab.research.google.com/github/PanagiotisChatzoglou/100-Days-Python/blob/main/PreSfhmmyDeployment_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

In [5]:
%env UID=683ef1e4c03ba2f496486d2f

env: UID=6530f9263773c5f7858b6b33


In [6]:
!apt update && apt upgrade && apt-get install -y software-properties-common redis-server
!service redis-server start

!pip install https://github.com/robotics-4-all/commlib-py/archive/devel.zip -Uq
!pip install https://github.com/robotics-4-all/goalee/archive/devel.zip -Uq

!git clone https://github.com/robotics-4-all/streamsim.git
!cd streamsim/ && git checkout tags/v0.7.0
!cd streamsim/ && pip3 install -r requirements.txt
!cd streamsim && pip3 install .

!bash -c '[ ! -z appmaker-executor ] && git clone https://github.com/robotics-4-all/appmaker-executor.git'
!bash -c '[ ! -z goal-dsl ] && git clone https://github.com/robotics-4-all/goal-dsl.git && cd goal-dsl && git checkout hackathon_2025 && pip install .'
!bash -c '[ ! -z goaldsl-executor ] && git clone https://github.com/robotics-4-all/goaldsl-executor.git && cd goaldsl-executor && git checkout hackathon_2025 && pip install -r requirements.txt'

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
4 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRe

# Start Executors

In [3]:
# Setup and Deploy required Executors
%env USE_REDIS=1

%env BROKER_TYPE=MQTT
%env BROKER_HOST=locsys.issel.ee.auth.gr
%env BROKER_PORT=8883
%env BROKER_SSL=True
%env BROKER_USERNAME=sensors
%env BROKER_PASSWORD=issel.sensors
%env HEARTBEATS=False
%env DEBUG=False
%env WAIT_FOR_EXECUTION_TERMINATION=False

%env GOALDSL_ZERO_LOGS=0
%env GOALDSL_LOG_LEVEL=CRITICAL
%env STREAMSIM_ZERO_LOGS=0
%env STREAMSIM_LOG_LEVEL=CRITICAL
%env APPMAKER_ZERO_LOGS=1
%env APPMAKER_LOG_LEVEL=CRITICAL


# Setup and Deploy StreamSim
!bash -c '[ -f streamsim.pid ] && pid=$(cat streamsim.pid) && kill $pid'
!nohup python3 streamsim/stream_simulator/bin/main.py ${UID} > streamsim.log 2>&1 & echo $! > streamsim.pid

# Setup and Deploy AppMaker-Executor
!bash -c '[ -f appexecutor.pid ] && pid=$(cat appexecutor.pid) && kill $pid'
!nohup python appmaker-executor/appmakerexecutor/src/appmakerexecutor/appmaker.py ${UID} > appexecutor.log 2>&1 & echo $! > appexecutor.pid

# Setup and Deploy GoalDSL-Executor
!bash -c '[ -f goalexecutor.pid ] && pid=$(cat goalexecutor.pid) && kill $pid'
!nohup python goaldsl-executor/executor.py > goalexecutor.log 2>&1 & echo $! > goalexecutor.pid

print("READY !!!")
# Check for errors
!tail -f goalexecutor.log appexecutor.log streamsim.log # | grep -E "error|warning|Error|Warning|ERROR|WARNING|Reconnecting in"

env: USE_REDIS=1
env: BROKER_TYPE=MQTT
env: BROKER_HOST=locsys.issel.ee.auth.gr
env: BROKER_PORT=8883
env: BROKER_SSL=True
env: BROKER_USERNAME=sensors
env: BROKER_PASSWORD=issel.sensors
env: HEARTBEATS=False
env: DEBUG=False
env: WAIT_FOR_EXECUTION_TERMINATION=False
env: GOALDSL_ZERO_LOGS=0
env: GOALDSL_LOG_LEVEL=CRITICAL
env: STREAMSIM_ZERO_LOGS=0
env: STREAMSIM_LOG_LEVEL=CRITICAL
env: APPMAKER_ZERO_LOGS=1
env: APPMAKER_LOG_LEVEL=CRITICAL
READY !!!
==> goalexecutor.log <==

==> appexecutor.log <==

==> streamsim.log <==
CRITICAL : stream_simulator.simulator : Simulator UID is: 6530f9263773c5f7858b6b33

==> appexecutor.log <==
Connected to broker
^C


# Kill All

In [4]:
!bash -c '[ -f streamsim.pid ] && pid=$(cat streamsim.pid) && kill $pid'
!bash -c '[ -f appexecutor.pid ] && pid=$(cat appexecutor.pid) && kill $pid'
!bash -c '[ -f goalexecutor.pid ] && pid=$(cat goalexecutor.pid) && kill $pid'